<a href="https://colab.research.google.com/github/ahlqui/VeloxChemColabs/blob/main/MolecularDynamicsConformerSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Generate a force field from XYZ and run dynamics


This notebook requires that you have a molecular structure. Two options:

Define Molecule as XYZ-coordinates: Can be generatred with free software such as www.avogadro.cc

Define Molecule with SMILES code, A molecule can be defined using a SMILES code (example below). We have two suggested ways to generate smiles from structure. 1) Sketch your molecule at https://www.rcsb.org/chemical-sketch and the SMILES code will be shown right below the structure. 2) Build your molecule at https://molview.org/ , go to Tools/Information Card and it will show you the SMILES code. Just copy/paste it into the SMILES box below.

Example of xyz-coordinates for CO2:

O 0.00 0.00 0.00

C 0.00 0.00 1.20

O 0.00 0.00 2.40

Example of SMILES for CO2:

O=C=O

In [1]:
#@title Install VeloxChem and dependencies (ca 1 min)
#@markdown Before you can run anything a few things need to be installed. It all runs on the Google Colab server an you do not need to install anything on your computer.
%%capture
import sys

!wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
!bash Mambaforge-Linux-x86_64.sh -bfp /usr/local
!mamba config --set auto_update_conda false
!mamba install --yes -c conda-forge -c veloxchem veloxchem openbabel py3Dmol mdanalysis openmm matplotlib

sys.path.append('/usr/local/lib/python3.10/site-packages/')

import veloxchem as vlx

In [2]:
xyz_coordinates = """
N     0.4785967928750  -0.4330741880750  -0.4934317502006
H     1.3794564278886   0.0429751463168  -0.5120458348956
C     0.4573285404340  -1.3154740355836  -1.6452845209868
H     1.2794157350200  -2.0270971721773  -1.5868316520123
H    -0.4697908768414  -1.8882598290096  -1.6558792536570
C     0.3747047426388  -1.2102491878280   0.7331072010194
H    -0.4082292852521  -1.9567463178277   0.6148319854093
H     1.2952953868059  -1.7540437197433   0.9183116804327
C     0.6956425554192   0.9059967227241  -5.3588904651214
C     1.6690431285267  -0.0704102447688  -5.0614336089256
C     1.5905279342425  -0.7922292640998  -3.8527793116028
C     0.5421211238334  -0.5430166236398  -2.9451176135833
C    -0.4312572408155   0.4285446612223  -3.2497814984180
C    -0.3564249048118   1.1570067527526  -4.4544417171718
H     0.7549521548685   1.4652126122086  -6.2812972715425
H     2.3338053657174  -1.5390048087124  -3.6163728176857
H    -1.2215586002930   0.6165771400284  -2.5371827271696
C    -0.5809783149118   1.1061887532548   4.2819306994214
C    -1.0539691464599  -0.2152180430193   4.1484967454886
C    -0.7402866542775  -0.9585904738513   2.9917812697427
C     0.0501603450518  -0.3918335577218   1.9715484114033
C     0.5329231859523   0.9234697295847   2.1220017776407
C     0.2096151223583   1.6793509886579   3.2655299818144
H    -0.8298576406810   1.6816374807225   5.1614537759970
H    -1.1155297675244  -1.9639771384900   2.8737258346041
H     1.1311551788061   1.3586475153088   1.3389498331805
C     2.7985412681099  -0.3440921389086  -6.0320361103160
H     3.0137177612643  -1.4119025048439  -6.0784904425001
H     2.5433990952730  -0.0090827377477  -7.0376742707821
H     3.7013243404231   0.1769923076871  -5.7132670703364
C    -1.3956677617432   2.2110713683443  -4.7713360805252
H    -2.2850215726603   2.0829577471538  -4.1532353918110
H    -0.9892606342753   3.2051959925139  -4.5854917335667
H    -1.6989550492144   2.1496453780473  -5.8166145479409
C    -1.9096053739985  -0.8269385079269   5.2372774019485
H    -1.8730938416183  -1.9157634078951   5.1986764605346
H    -1.5629099527925  -0.5125717843390   6.2218512212214
H    -2.9465170241487  -0.5126306418706   5.1188520156529
C     0.7055071531436   3.1039291474455   3.3878074482962
H     0.6996321098764   3.5926125742168   2.4126899821207
H     0.0720511304863   3.6841458846608   4.0587668370105
H     1.7245462284362   3.1151446899290   3.7742478882927
"""


In the box below you can input the smiles code of the molecule. If you want to generate forcefields for molecular dynamics, enter the name that you want for your files and a three letter residue name. This code will generate a GAFF2 force field with RESP-charges calculated at the HF/sto-3g level of theory. Since this is a quantum chemical computations it may take some time if your molecule is larger.

(If you want a different basis set you can change the basis = vlx.MolecularBasis.read(molecule, 'sto-3g') line in the "Generate a force field" cell below)

In [3]:
#@title Define your molecule, either asa SMILES code or xyz-coordinates above
#@markdown - Enter the SMILES code
smiles_code = 'OCCC=O' #@param {type:"string"}
#@markdown - If you want to use the xyz coordinates in the previous cell check box
use_xyz = False # @param {type:"boolean"}
if use_xyz == False:
  molecule = vlx.Molecule.read_smiles(smiles_code)
else:
  molecule = vlx.Molecule.read_str(xyz_coordinates)
print('Structure of the molecule entered: ')
molecule.show(atom_indices=True)
#@markdown - Give your Gromacs files a name
gromacs_file = 'test' #@param {type:"string"}
#@markdown - Give your molecule a residue name (three letters)
residue_name = 'tes' #@param {type:"string"}



Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
#@title Generate a force field and write Gromacs files
#@markdown - Run this cell to generate the force field
#@markdown - Example timing: Ibuprofen (1 min)
#@markdown - The files can be found on the left panel ("Files")
ff_gen = vlx.ForceFieldGenerator()
basis = vlx.MolecularBasis.read(molecule, 'sto-3g')
ff_gen.ostream.mute()
ff_gen.create_topology(molecule, basis)

ff_gen.write_gromacs_files(gromacs_file, residue_name)


In [5]:
#@title Run dynamics
#@markdown - Example timings: 200 ps MD of ibuprofen (22 sec)
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as mda
universe = mda.Universe(gromacs_file + '.gro')
with mda.Writer(gromacs_file + '.gro') as gro:
    gro.write(universe)
gro = GromacsGroFile(gromacs_file + '.gro')
top = GromacsTopFile(gromacs_file + '.top')
system = top.createSystem(nonbondedMethod=NoCutoff)
#@markdown - Set the timestep in femtoseconds
#time_step = 2 #@param {type:"integer"}
time_step = 2 #@param [1, 2]
#@markdown - Set the temperature in Kelvin
temp = 700 #@param {type:"slider", min:200, max:700, step:1}
integrator = LangevinMiddleIntegrator(temp*kelvin, 1/picosecond, 0.001*time_step*picoseconds)
simulation = Simulation(top.topology, system, integrator)
simulation.context.setPositions(gro.positions)
simulation.minimizeEnergy()
#@markdown - Set the number of steps you want to take (ex 100000 steps * 2 fs= 200 ps)
steps = 100000 #@param {type:"integer"}
#@markdown - Number of snapshots saved. Select how many structures you want to save from your dynamics run
snapshots = 10 #@param {type:"integer"}
outputdata = steps / snapshots
simulation.reporters.append(PDBReporter(gromacs_file + '.pdb', outputdata))
simulation.reporters.append(StateDataReporter(stdout, outputdata, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(steps)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10000,61.95976448059082,687.1787825055869
20000,10.097129821777344,651.9286394993496
30000,-2.1937637329101562,508.6220681362917
40000,-23.63544464111328,435.9027239712622
50000,-9.499794006347656,854.0721998611559
60000,38.10314178466797,473.4716313038199
70000,34.506134033203125,735.5519652030478
80000,27.864906311035156,416.05815582453926
90000,38.40420150756836,618.0853446046658
100000,27.479568481445312,592.29098830658


In [6]:
#@title Visualize the dynamics
import py3Dmol as p3d
interval = 400 #@param  {type:"slider", min:100, max:2000, step:100}
print('Output geometry:')
viewer = p3d.view(width=400,height=250)
viewer.addModelsAsFrames(open(gromacs_file + '.pdb', 'r').read(),'pdb', {'keepH': True})
viewer.animate({'interval': interval, 'loop': "forward", 'reps': 10})
viewer.setStyle({"stick":{},"sphere": {"scale":0.25}})
viewer.zoomTo()
viewer.show()

Output geometry:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
#@title MM optimization of all snapshots
#@markdown 10 snapshots of ibuprofen (3 min)
import MDAnalysis as mda

for i in range(snapshots):
  universe = mda.Universe(gromacs_file + '.pdb')
  universe.trajectory[i]
  allatoms = universe.select_atoms('all')
  allatoms.write(gromacs_file + str(i) + '.xyz')
  #with mda.Writer(gromacs_file + '.xyz') as xyz:
  #   xyz.write(universe)

total_results = []
for i in range(snapshots):
  molecule = vlx.Molecule.read_xyz_file(gromacs_file + str(i) + '.xyz')
  print('Optimizing structure ' + str(i+1) + ' of ' + str(snapshots))
  vlxopenmm = vlx.OpenMMDriver()
  vlxopenmm.add_topology(gromacs_file + '.top')
  opt = vlx.OptimizationDriver(vlxopenmm)
  opt.ostream.mute()
#    opt.constraints = ["freeze dihedral 18 14 12 11"]
  results = opt.compute(molecule)

  total_results.append(results)

print(' ')

last_energies = []
final_geometries = []

for i in range(snapshots):
  res = total_results[i]['opt_energies'][-1]
  last_energies.append(res)
for i in range(snapshots):
  geos = total_results[i]['final_geometry']
  final_geometries.append(geos)
for i in range(snapshots):
  print('Energy', last_energies[i])

m = min(last_energies)
m_index = last_energies.index(m)
print(final_geometries[m_index])
lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[m_index])
lowest_geometry.show()

Optimizing structure 1 of 10
Optimizing structure 2 of 10
Optimizing structure 3 of 10
Optimizing structure 4 of 10
Optimizing structure 5 of 10
Optimizing structure 6 of 10
Optimizing structure 7 of 10
Optimizing structure 8 of 10
Optimizing structure 9 of 10
Optimizing structure 10 of 10
 
Energy -0.020511238446097074
Energy -0.01870079202875999
Energy -0.019498619616859402
Energy -0.020642427435929152
Energy -0.020642427435929152
Energy -0.01949829706385645
Energy -0.02030935513684279
Energy -0.02030936094860861
Energy -0.01949862252274231
Energy -0.02030936385449152
11

O              1.168436569135        -1.504666609453         1.327503031726
C              2.500486805522        -1.022324646042         1.405634550473
C              2.726456723076        -0.035133898314         0.272539328936
C              4.119034180122         0.538247372057         0.266668447882
O              4.985772007140         0.245097398015         1.086346025124
H              1.039537813275        -2

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
#@title XTB optimization of all snapshots
#@markdown Example timing: ibuprofen (7 min)
import MDAnalysis as mda


for i in range(snapshots):
  universe = mda.Universe(gromacs_file + '.pdb')
  universe.trajectory[i]
  allatoms = universe.select_atoms('all')
  allatoms.write(gromacs_file + str(i) + '.xyz')
  #with mda.Writer(gromacs_file + '.xyz') as xyz:
  #   xyz.write(universe)

total_results = []
for i in range(snapshots):
  print('Optimizing structure ' + str(i+1) + ' of ' + str(snapshots))
  molecule = vlx.Molecule.read_xyz_file(gromacs_file + str(i) + '.xyz')
  vlxXtb = vlx.XtbDriver()
  opt = vlx.OptimizationDriver(vlxXtb)
  opt.ostream.mute()
#    opt.constraints = ["freeze dihedral 18 14 12 11"]
  results = opt.compute(molecule)

  total_results.append(results)

print(' ')
last_energies = []
final_geometries = []

for i in range(snapshots):
  res = total_results[i]['opt_energies'][-1]
  last_energies.append(res)
for i in range(snapshots):
  geos = total_results[i]['final_geometry']
  final_geometries.append(geos)
for i in range(snapshots):
  print('Energy', last_energies[i])

m = min(last_energies)
m_index = last_energies.index(m)
print(final_geometries[m_index])
lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[m_index])
lowest_geometry.show()

Optimizing structure 1 of 10
Optimizing structure 2 of 10
Optimizing structure 3 of 10
Optimizing structure 4 of 10
Optimizing structure 5 of 10
Optimizing structure 6 of 10
Optimizing structure 7 of 10
Optimizing structure 8 of 10
Optimizing structure 9 of 10
Optimizing structure 10 of 10
 
Energy -17.57968149976963
Energy -17.574244077816452
Energy -17.57572625610876
Energy -17.576901337626985
Energy -17.576900768678374
Energy -17.575726256191874
Energy -17.575129890752745
Energy -17.575129969869014
Energy -17.57572625715648
Energy -17.575661940011496
11

O              2.223897074756        -0.661779725483        -0.623206602709
C              1.950613843878        -0.113606936425         0.636167802777
C              2.675147817037        -0.853954811806         1.770400579414
C              4.167213114537        -0.782599072199         1.637922536664
O              4.759680372475        -0.363330184780         0.681914816403
H              3.172861704294        -0.561504461920    

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
#@title Print relative energies of all structures
for i in range(snapshots):
  lowest_geometry = vlx.Molecule.read_xyz_string(final_geometries[i])
  print('Relative energy: ', (last_energies[i] - last_energies[m_index])*627.5)
  lowest_geometry.show()
  print(final_geometries[i])

Relative energy:  0.0


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

11

O              2.223897074756        -0.661779725483        -0.623206602709
C              1.950613843878        -0.113606936425         0.636167802777
C              2.675147817037        -0.853954811806         1.770400579414
C              4.167213114537        -0.782599072199         1.637922536664
O              4.759680372475        -0.363330184780         0.681914816403
H              3.172861704294        -0.561504461920        -0.783555233502
H              0.869322334740        -0.191695000188         0.774757789978
H              2.231706612355         0.949164916246         0.668132672826
H              2.403355035221        -1.914035304606         1.752609798158
H              2.380062036386        -0.455501332795         2.742616599624
H              4.712140054322        -1.176158086045         2.518239240368

Relative energy:  3.4119822756196694


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

11

O              1.731477151026        -0.560952130162         2.197610139269
C              1.610419431637        -0.579226731730         0.793336306113
C              2.991784522137        -0.845102254616         0.204900830005
C              3.966688936066         0.253409950083         0.529553230349
O              5.150078094371         0.113722982887         0.637391737255
H              0.865650145558        -0.423846498110         2.594763718370
H              1.228569181841         0.385559998723         0.425702632476
H              0.916802925782        -1.370144847624         0.474084949717
H              2.916386855474        -0.907813281820        -0.883837734308
H              3.385619724008        -1.791621626191         0.577310585364
H              3.493523032100         1.248014438561         0.647183605389

Relative energy:  2.4819153971973673


...

11

O              3.140506360235        -1.745522721089         2.321913385075
C              2.631698483958        -0.434414315038         2.220084935648
C              3.587349921812         0.388729966960         1.355927581237
C              3.615379479278        -0.106598509526        -0.062218426094
O              3.630665990193         0.590208418892        -1.036829564738
H              2.535980638653        -2.284565002973         2.841447553019
H              2.556312079184         0.028416594453         3.214638972896
H              1.630869531532        -0.439417628221         1.764785662202
H              4.597233831333         0.264716142237         1.756403996241
H              3.320534498018         1.445452592426         1.382763123861
H              3.630469185804        -1.209005538122        -0.134917219347

Relative energy:  2.481914739752984


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

11

O              3.855772653806         0.780854464778        -0.994444842306
C              2.939529675524         0.433041496956         0.008714400733
C              3.570441144858        -0.435636173846         1.106366677267
C              2.587080045706        -0.778915220003         2.186138161103
O              2.448617731359        -1.859126254291         2.681895295755
H              4.578209291460         1.288632540139        -0.607379247327
H              2.151121274272        -0.125374746669        -0.500882622119
H              2.494466334169         1.332370297226         0.460589472013
H              4.383731033220         0.125902436470         1.579957198541
H              3.987723731412        -1.347684392817         0.676810076699
H              1.975307084214         0.085935552058         2.515235429641

Relative energy:  2.522273748229402


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

11

O              1.337769505339        -0.239812083171        -0.099733982858
C              2.144978909249         0.614645944175         0.665104745042
C              3.009029886203        -0.125077148280         1.708073201149
C              4.080977496380        -0.943115690029         1.053144255689
O              5.259835118621        -0.793757183630         1.203598905256
H              1.885860941579        -0.751941229699        -0.703800947034
H              1.451401538743         1.275260632696         1.190958491036
H              2.791855875026         1.229075320774         0.024751674328
H              2.358840804709        -0.795246140642         2.274692417642
H              3.462984586786         0.602298356866         2.381231472459
H              3.691465337364        -1.734330779059         0.378979767290

